In [73]:
import pandas as pd
import numpy as np
import json
import random
import itertools
import ast
from collections import defaultdict
%matplotlib inline

In [74]:
dump_fpath = 'data/declarations.json'

with open(dump_fpath) as f:
    declarations = json.loads(f.read())

In [75]:
random.choice(declarations)

{'bonds': [],
 'incomes': [{'comment': '', 'relative': None, 'size': 288208.0}],
 'main': {'document_type': {'id': 1, 'name': 'Антикоррупционная декларация'},
  'office': {'id': 1417,
   'name': 'Советский - городской округ',
   'post': 'Ведущий специалист',
   'region': {'id': 46, 'name': 'Калининградская область'},
   'type': {'id': 20, 'name': 'Муниципальный, без структуры'},
   'url': 'http://www.sovetsk-tilsit.ru/'},
  'party': None,
  'person': {'family_name': 'Парусимова',
   'given_name': 'Ирина',
   'id': 4094,
   'name': 'Парусимова Ирина Владиславовна',
   'patronymic_name': 'Владиславовна'},
  'year': 2012},
 'real_estates': [{'comment': '',
   'country': 'Россия',
   'name': '',
   'own_type': {'id': 1, 'name': 'В собственности'},
   'region': None,
   'relative': None,
   'share': None,
   'share_type': {'id': 3, 'name': 'Совместная собственность'},
   'square': 71.5,
   'type': {'id': 4, 'name': 'Квартира'}}],
 'savings': [],
 'spendings': [],
 'stocks': [],
 'vehicles':

Total declarations:

In [76]:
len(declarations)

90864

In [77]:
columns = ['person_id', 'person_name', 'year', 'office_id',
           'office_name', 'income_personal', 'income_relatives']

rows = []
for dec in declarations:
    row = [
        dec['main']['person']['id'],
        dec['main']['person']['name'],
        dec['main']['year'],
        dec['main']['office']['id'],
        dec['main']['office']['name'],
    ]
    income_personal_sum = sum([inc['size'] for inc in dec['incomes'] if inc['relative'] is None])
    income_relatives_sum = sum([inc['size'] for inc in dec['incomes'] if inc['relative'] is not None])
    # savings_sum = sum([float(sav.split('руб.')[0].replace(',', '.').replace(' ', '')) for sav in dec['savings']])
    # real_estate_amount = len(dec['real_estates'])
    # real_estate_squares_sum = sum([(estate['square']  or 0) for estate in dec['real_estates']])
    row += [income_personal_sum, income_relatives_sum]
    rows.append(row)
len(rows)

90864

In [78]:
df = pd.DataFrame(rows, columns=columns)
df = df.sort_values(by=['person_id', 'year'])
df.head()

,person_id,person_name,year,office_id,office_name,income_personal,income_relatives
30358,8,Зюганов Геннадий Андреевич,1998,14,Государственная Дума,124154.00,0.00
31970,8,Зюганов Геннадий Андреевич,1999,449,Президент Российской Федерации,495443.00,10804.00
127,8,Зюганов Геннадий Андреевич,2006,14,Государственная Дума,1257784.00,0.00
1109,8,Зюганов Геннадий Андреевич,2006,449,Президент Российской Федерации,3445291.61,145376.87
5520,8,Зюганов Геннадий Андреевич,2009,14,Государственная Дума,2075542.33,102279.19


Total unique persons

In [79]:
unique_persons_count = df['person_id'].unique().shape[0]
print(unique_persons_count)

51909


Years

In [80]:
print(df.year.min(), df.year.max())

1998 2017


In [81]:
df['year'].value_counts()

2014    24048
2015    20302
2016    19608
2013    12360
2012     5547
2011     4063
2010     3142
2009     1327
2006      280
2008       86
1998       81
1999       12
2017        8
Name: year, dtype: int64

Remove all declarations from years 1998 - 2012, 2017.

In [82]:
df = df[(df.year >= 2013) & (df.year <= 2016)]
df.head()

,person_id,person_name,year,office_id,office_name,income_personal,income_relatives
7919,8,Зюганов Геннадий Андреевич,2013,14,Государственная Дума,3194615.67,198267.22
16484,8,Зюганов Геннадий Андреевич,2014,14,Государственная Дума,4460788.41,190086.74
31525,8,Зюганов Геннадий Андреевич,2015,14,Государственная Дума,6538890.04,183815.19
56776,8,Зюганов Геннадий Андреевич,2016,14,Государственная Дума,5499187.78,187567.64
7643,9,Алферов Жорес Иванович,2013,14,Государственная Дума,16647820.37,1485698.88


Declarations count after filtering

In [101]:
df.shape

(76318, 8)

Unique persons after filtering

In [83]:
unique_persons_count = df['person_id'].unique().shape[0]
print(unique_persons_count)

49032


In [84]:
df.to_csv('data/declarations.csv', index=False)

Count number of times persons worked at the same office in the same year

In [85]:
grouped_view = df.groupby(['person_id', 'person_name', 'year']).first()

In [86]:
common_offices = df.groupby(['year', 'office_id'])['person_id'].apply(list).reset_index()
common_offices.head()

,year,office_id,person_id
0,2013,1,"[294, 296, 298, 1783, 2477, 2478, 2811, 9129, ..."
1,2013,2,"[6946, 6947, 6948, 6949, 6950, 6951, 6952, 695..."
2,2013,3,"[679, 680, 681, 682, 7300, 7301, 7302, 7304, 7..."
3,2013,4,"[5350, 11677, 11685, 11686, 11687, 11688, 1168..."
4,2013,5,"[34, 114, 139, 180, 190, 231, 234, 236, 299, 4..."


In [87]:
common_offices.to_csv('data/common_offices.csv', index=False)

## Граф связей

In [88]:
YEARS = [2013, 2014, 2015, 2016]

In [89]:
common_offices_filtered = common_offices[common_offices['year'].isin(YEARS)].copy()
common_offices_filtered.head()

,year,office_id,person_id
0,2013,1,"[294, 296, 298, 1783, 2477, 2478, 2811, 9129, ..."
1,2013,2,"[6946, 6947, 6948, 6949, 6950, 6951, 6952, 695..."
2,2013,3,"[679, 680, 681, 682, 7300, 7301, 7302, 7304, 7..."
3,2013,4,"[5350, 11677, 11685, 11686, 11687, 11688, 1168..."
4,2013,5,"[34, 114, 139, 180, 190, 231, 234, 236, 299, 4..."


In [90]:
def populate_dict(person_id_list, graph_dict):
    for pair in itertools.combinations(person_id_list, 2):
        graph_dict[pair] += 1
worked_together_dict = defaultdict(int)    
common_offices_filtered['person_id'].map(lambda person_id_list: populate_dict(person_id_list, worked_together_dict))
len(worked_together_dict)

4163369

In [91]:
len(worked_together_dict)

4163369

In [92]:
# More or equal to the amount of persons
# Less than number of all possible pairs
assert unique_persons_count <= len(worked_together_dict) <= unique_persons_count*(unique_persons_count-1)/2 

In [100]:
with open('data/graph.csv', 'w') as graph_file:
    graph_file.write('person1,person2,times_worked_together\n')
    for pair in sorted(worked_together_dict):
        graph_file.write('{},{},{}\n'.format(pair[0], pair[1], worked_together_dict[pair]))

## Справочник доходов депутатов

In [96]:
df['income'] = df['income_personal'] + df['income_relatives']

In [97]:
# Если несколько записей на одного депутата на один год на один офис, то берем первую
officials_year = (df
                    .groupby(by=['year', 'person_id', 'person_name', 'office_id'])['income']
                    .first()
                    .to_frame('income')
                    .reset_index())
officials_year.head()

,year,person_id,person_name,office_id,income
0,2013,8,Зюганов Геннадий Андреевич,14,3392882.89
1,2013,9,Алферов Жорес Иванович,14,18133519.25
2,2013,10,Андреев Андрей Анатольевич,14,4368530.71
3,2013,12,Афонин Юрий Вячеславович,14,2802081.80
4,2013,13,Гаврилов Сергей Анатольевич,14,3263364.19


In [98]:
# Суммируем доход в разных офисах
officials_income = (officials_year
                    .groupby(by=['person_id', 'person_name'])['income']
                    .sum()                    
                    .to_frame('income')
                    .reset_index())
officials_income = officials_income.set_index('person_id')
officials_income.head()

,person_name,income
person_id,,
8,Зюганов Геннадий Андреевич,20453218.69
9,Алферов Жорес Иванович,81462661.75
10,Андреев Андрей Анатольевич,14475607.50
12,Афонин Юрий Вячеславович,23774458.69
13,Гаврилов Сергей Анатольевич,19608057.29


In [103]:
officials_income.to_json('data/officials_income.json', orient='index', force_ascii=False, double_precision=2)
officials_income.to_csv('data/officials_income.csv')